In [1]:
import os, json, time
from datetime import datetime

In [2]:
from dotenv import load_dotenv
dotenv_path = '../.env'
load_dotenv(dotenv_path)

True

In [3]:
from db_connect_mag_201710 import get_db_connection

In [4]:
db = get_db_connection()

/home/jporteno/code/maka/venv/lib/python3.5/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [5]:
tbl = db.tables['Papers']

In [6]:
# http://docs.sqlalchemy.org/en/latest/orm/extensions/automap.html
from sqlalchemy.ext.automap import automap_base
Base = automap_base()
Base.prepare(db.engine, reflect=True)

In [7]:
Paper = Base.classes.Papers
PaperAuthorAffiliation = Base.classes.PaperAuthorAffiliations
PaperReference = Base.classes.PaperReferences

In [8]:
pr = PaperReference()

In [9]:
from sqlalchemy.orm import Session

In [10]:
session = Session(db.engine)

In [29]:
fname = '../paperscrape4/papers-1960.json'
with open(fname, 'r') as f:
    j = json.load(f)

In [30]:
j[0]

{'AA': [{'AfId': None, 'AuId': 2308212464, 'AuN': 'r g d steel'},
  {'AfId': None, 'AuId': 2314176326, 'AuN': 'james h torrie'}],
 'C': None,
 'D': '1960-01-01',
 'E': {'DOI': None},
 'F': [{'FId': 48057537},
  {'FId': 145420912},
  {'FId': 149782125},
  {'FId': 105795698},
  {'FId': 127413603}],
 'Id': 1992897439,
 'J': None,
 'L': 'en',
 'RId': None,
 'Ti': 'principles and procedures of statistics',
 'Y': 1960}

In [31]:
from collections import Counter
c = Counter()
for record in j:
    journal = record.get('C')
    if not journal:
        journal = []
    if len(journal) == 2:
        break
    c[len(journal)] += 1
c

Counter({0: 853})

In [32]:
journal

{'CId': 2755067489, 'CN': 'ire'}

In [33]:
def process_paper(record):
    p = Paper()
    p.Paper_ID = record.get('Id')
    p.title = record.get('Ti')
    p.date = record.get('D')
    p.year = record.get('Y')
    journal = record.get('J')
    if journal:
        p.Journal_ID = journal.get('JId')
    conference = record.get('C')
    if conference:
        p.Conference_series_ID = conference.get('CId')
    extended = record.get('E')
    if extended:
        p.DOI = extended.get('DOI')
        
    prs = []
    references = record.get('RId')
    if references:
        for rid in references:
            pr = PaperReference()
            pr.Paper_ID = p.Paper_ID
            pr.Paper_reference_ID = rid
            prs.append(pr)
    
    paas = []
    authors = record.get('AA')
    if authors:
        for a in authors:
            paa = PaperAuthorAffiliation()
            paa.Author_ID = a.get('AuId')
            paa.Paper_ID = p.Paper_ID
            paa.Author_name = a.get('AuN')
            paa.Affiliation_ID = a.get('AfId')
            paas.append(paa)
    
    return p, prs, paas

In [34]:
for record in j:
    p, prs, paas = process_paper(record)
    if prs:
        break

In [35]:
paas[0].Author_ID

2275121949

In [36]:
paas[0].Author_name

'r e kalman'

In [37]:
record

{'AA': [{'AfId': 40036882, 'AuId': 2275121949, 'AuN': 'r e kalman'}],
 'C': None,
 'D': '1960-01-01',
 'E': {'DOI': '10.1115/1.3662552'},
 'F': [{'FId': 65244806}, {'FId': 126255220}, {'FId': 105795698}],
 'Id': 2105934661,
 'J': None,
 'L': 'en',
 'RId': [2013548638,
  2062328787,
  2075549445,
  610132389,
  642684674,
  2571506285,
  1999276251,
  2112172398,
  71817473,
  2031003381,
  2023026992,
  2076492098,
  2115971087,
  2044144205,
  2107219746,
  2030291784,
  1994643457],
 'Ti': 'a new approach to linear filtering and prediction problems',
 'Y': 1960}

In [38]:
for pr in prs:
    print(pr.Paper_reference_ID)

2013548638
2062328787
2075549445
610132389
642684674
2571506285
1999276251
2112172398
71817473
2031003381
2023026992
2076492098
2115971087
2044144205
2107219746
2030291784
1994643457


In [39]:
# try loading this one
session.add(p)
for pr in prs:
    session.add(pr)
for paa in paas:
    session.add(paa)
session.commit()

In [11]:
fname = '../paperscrape4/papers-1960.json'

In [12]:
json_load = Base.classes['paperjson_loaded']()

In [13]:
json_load.json_fname = os.path.basename(fname)

In [14]:
json_load.load_start = datetime.now()
time.sleep(2)
json_load.load_end = datetime.now()

In [15]:
json_load.num_records = 55555
json_load.num_added = 55555

In [16]:
session.add(json_load)
session.commit()

In [17]:
Base.classes.has_key('paperjson_loaded')

True

In [21]:
DBLog = Base.classes['paperjson_loaded']

In [22]:
q = session.query(DBLog).filter(DBLog.json_fname=='papers-1960.json')
# q = q.exists()
q = q.scalar()

In [23]:
q is not None

False

In [24]:
session.close()